In [ ]:
# only run this on Google Colab
!pip install geopandas
!pip install folium

In [ ]:
import pandas as pd
import seaborn as sns
import folium
import numpy as np
import matplotlib.pyplot as plt

# Load

In [ ]:
main_df = pd.read_csv('https://raw.githubusercontent.com/Dynovski/BESTHL-AI/main/train_data.csv?token=GHSAT0AAAAAABTLWN7I6LMAI3BK5ZSQSE7WYSZQ4DA', error_bad_lines=False)
print(main_df.head())

In [ ]:
processed_df = pd.read_csv('processed.csv')
processed_df.head()

# Basic stuff

In [ ]:
main_df.shape

In [ ]:
main_df.columns

In [ ]:
main_df.info()

In [ ]:
main_df.describe()

In [ ]:
main_df['POSTCODE'].value_counts()

In [ ]:
sns.scatterplot(data=main_df, x=main_df.index, y=main_df["FULLVAL"])

In [ ]:
type(main_df['TAXCLASS'][0])

In [ ]:
main_df['TAXCLASS'].head(19)

In [ ]:
sns.catplot(data=main_df, x=main_df.index, y=main_df["TAXCLASS"])

# Map

In [ ]:
main_df['Latitude'].value_counts().head(30)

In [ ]:
main_df['Longitude'].value_counts().head(30)

In [ ]:
# Watch out! sampling resets index
map_df = main_df[main_df['Latitude'].notna()].sample(frac=1).reset_index(drop=True)

# using too big location_number param results in slow map 
location_number = 1000

map_df = map_df[map_df['Longitude'].notna()].head(location_number)
print(map_df.shape)

In [ ]:
map_df['Latitude'].loc[0]
type(map_df['Latitude'].loc[0])

In [ ]:
map_df['Longitude'].loc[0]
type(map_df['Latitude'].loc[0])

In [ ]:
map_full = folium.Map(location=[40.719232, -74.00809], zoom_start=9)

In [ ]:
for index, _ in map_df.iterrows():
    folium.Marker([map_df['Latitude'].loc[index], map_df['Longitude'].loc[index]]).add_to(map_full)

In [ ]:
map_full

## Separating clusters

### k-means

In [ ]:
from scipy.cluster.vq import kmeans2, whiten

In [ ]:
coordinates_kmeans = [[map_df['Latitude'].loc[index], map_df['Longitude'].loc[index]] for index, _ in map_df.iterrows()]
coordinates_kmeans = np.array(coordinates_kmeans)
print(type(coordinates_kmeans))
print(len(coordinates_kmeans))
print(coordinates_kmeans[19])
print(type(coordinates_kmeans[19]))

In [ ]:
x, y = kmeans2(whiten(coordinates_kmeans), 5, iter = 20)  
plt.scatter(coordinates_kmeans[:,0], coordinates_kmeans[:,1], c=y)
plt.show()

### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [ ]:
coordinates_dbscan = map_df[['Latitude', 'Longitude']].to_numpy()

In [ ]:
kms_per_radian = 6371.0088
epsilon = 1.7 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree', metric='haversine').fit(np.radians(coordinates_dbscan))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coordinates_dbscan[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

In [ ]:
print(clusters.shape)
print(clusters.info())
print(type(clusters[2]))
print(len(clusters[2]))

In [ ]:
map_dbscan = folium.Map(location=[40.719232, -74.00809], zoom_start=9)

In [ ]:
for place in clusters[0]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="orange")).add_to(map_dbscan)

In [ ]:
for place in clusters[1]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="blue")).add_to(map_dbscan)

In [ ]:
for place in clusters[2]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="purple")).add_to(map_dbscan)

In [ ]:
for place in clusters[3]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="green")).add_to(map_dbscan)

In [ ]:
for place in clusters[4]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="red")).add_to(map_dbscan)

In [ ]:
for place in clusters[5]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="gray")).add_to(map_dbscan)

In [ ]:
map_dbscan

# Augment

In [ ]:
augmented_df = processed_df

In [ ]:
augmented_df.loc[(augmented_df['Latitude'] <= 90) & (augmented_df['Latitude'] >= -90)]

In [ ]:
augmented_df.loc[(augmented_df['Longitude'] <= 180) & (augmented_df['Longitude'] >= -180)]

In [ ]:
augmented_df['1-closest neighbour value'] = 0
augmented_df['2-closest neighbour value'] = 0
augmented_df['3-closest neighbour value'] = 0
augmented_df['4-closest neighbour value'] = 0
augmented_df['5-closest neighbour value'] = 0

augmented_df[['1-closest neighbour value', 
              '2-closest neighbour value', 
              '3-closest neighbour value', 
              '4-closest neighbour value', 
              '5-closest neighbour value']].head()

In [ ]:
from sklearn.neighbors import BallTree

In [ ]:
print(augmented_df[['Latitude', 'Longitude']].values.shape)

In [ ]:
tree = BallTree(np.deg2rad(augmented_df[['Latitude', 'Longitude']].values), metric='haversine')

In [ ]:
query_lats = augmented_df['Latitude']
query_lons = augmented_df['Longitude']
print(query_lats.shape)
print(query_lons.shape)
print(query_lats.head())
print(query_lons.head())

In [ ]:
# Find closest city in reference dataset for each in df_other
# use k = 6 for 5 closest neighbors + self
distances, indices = tree.query(np.deg2rad(np.c_[query_lats, query_lons]), k = 6)
# print(len(distances))
# print(len(indices))

r_km = 6371 # multiplier to convert to km (from unit distance)
for name, d, ind in zip(augmented_df.index, distances, indices):
    print(f"Property with index {name} closest matches:")
    for i, index in enumerate(ind):
        print(f"\t{index} with distance {d[i]*r_km:.4f} km")

In [ ]:
distances, indices = tree.query(np.deg2rad(np.c_[query_lats, query_lons]), k = 6)
helper_df = augmented_df['FULLVAL']

r_km = 6371 # multiplier to convert to km (from unit distance)
for work_index, d, ind in zip(augmented_df.index, distances, indices):
    for i, index in enumerate(ind):
        if i:
            augmented_df.loc[work_index, f'{i}-closest neighbour value'] = helper_df.loc[index]

In [ ]:
augmented_df[['1-closest neighbour value', 
              '2-closest neighbour value', 
              '3-closest neighbour value', 
              '4-closest neighbour value', 
              '5-closest neighbour value']].head()

In [ ]:
print(augmented_df.shape)
print(augmented_df.columns)

In [ ]:
augmented_df.to_csv('./augmented.csv', index=False)

In [ ]:
test_df = pd.read_csv('augmented.csv')
print(test_df.shape)
print(test_df.columns)